# SQL Connection and main imports

In this section, we'll declare a class called Query_Builder, wich is responsible to gather all query functions, as well as SQL connection

In [9]:
import pandas as pd
import sqlite3

#Class responsible to create and manage all Queries and SQL connection
class Query_Builder:
  #Creating Connection
  def connect():
    conn = sqlite3.connect('base_bacen.sqlite')
    return conn

  #Function to execute queries
  def query(self, conn, sql_query):
    try:
        df = pd.read_sql(sql_query, con=conn)
    except TypeError:
        df = None
    return df
  
  #Function to find NULL by one column
  def select_null(self, column, table):
    query_select_null = 'SELECT ' + column + ' ' + \
      'FROM ' + table + ' ' + \
      'WHERE ' + column + ' = "nan" ' + \
      'OR ' + column + ' = NULL'
    return query_select_null
  
  #Function to find all table NULLs 
  def verify_all_table_nulls(self, list_columns, table):
    for column in list_columns:
      select_null_by_column = self.select_null(column, table)
      null_columns = self.query(self.conn, select_null_by_column)
      if (null_columns.empty):
        print("Column " + column + ": 0")
      else:
        print("Column " + column + ": " + str(len(null_columns)))

  #Function that selects Mode of a column
  def select_mode(self, column, table):
    mode = 'SELECT ' + column + ' ' + \
      'AS ' + 'mode_' + column + ' ' + \
      'FROM ' + table + ' ' \
      'GROUP BY ' + column + ' ' + \
      'ORDER  BY COUNT(*) DESC ' + \
      'LIMIT 1'
    return mode

  #Function that selects Mean of a column
  def select_mean(self, column, table):
    mean = 'SELECT AVG(' + column + ') ' + \
      'as ' + 'mean_' + column + ' ' + \
      'FROM ' + table
    return mean

  #Function that selects Median of a column
  def select_median(self, column, table):
    median = 'SELECT AVG(' + column + ') ' + \
     'AS ' + 'median_' + column + ' ' + \
      'FROM (SELECT ' + column + ' ' + \
      'FROM ' + table + ' ' + \
        'ORDER BY ' + column + ' ' + \
         'LIMIT 2 - (SELECT COUNT(*) FROM ' + table + ') % 2 ' + \
      'OFFSET (SELECT (COUNT(*) - 1) / 2 ' + \
        'FROM ' + table + '))'
    return median

  #Function to creat CASE query
  def case_query(self, prd, prd_dictionary, list_columns, prd_qt):
    final_query = ''
    or_query = ''
    match(prd_qt):
      case 2:
        or_query = 'OR codigo_modalidade_operacao = ' + str(prd_dictionary[prd + '_1']) + ' '
      case 3:
        or_query = 'OR codigo_modalidade_operacao = ' + str(prd_dictionary[prd + '_1']) + ' ' + \
         'OR codigo_modalidade_operacao = ' + str(prd_dictionary[prd + '_2']) + ' '
      case 4:
        or_query = 'OR codigo_modalidade_operacao = ' + str(prd_dictionary[prd + '_1']) + ' ' + \
         'OR codigo_modalidade_operacao = ' + str(prd_dictionary[prd + '_2']) + ' ' + \
         'OR codigo_modalidade_operacao = ' + str(prd_dictionary[prd + '_3']) + ' '

    for column in list_columns:
      query_cheque_especial = 'CASE ' + \
        'WHEN codigo_modalidade_operacao = ' + str(prd_dictionary[prd]) + ' ' + \
        or_query + \
        'THEN ' +  str(column) + ' ' + \
        'ELSE 0 ' + \
        'END as "' + str(prd) + '_' + str(column) + '" , '
      final_query += query_cheque_especial
    return final_query[:-2]

  #Function to create SUM
  def create_SUM(self, list_prd, list_columns):
    final_SUM = ''
    for prd in list_prd:
      for column in list_columns:
        column_name = str(prd) + '_' + str(column)
        aux_SUM = 'SUM(' + column_name + ') as ' + column_name + ', '
        final_SUM += aux_SUM
    return final_SUM[:-2]

  #Function to create 0_60 credito_vencer interval
  def credito_vencer_interval_0_60(self, column_start, column_end, prd):
    column_start = prd + '_' + column_start
    column_end = prd + '_' + column_end
    interval = 'SUM(' + column_start + ' + ' + column_end + ') as ' + prd + '_credito_vencer_0_60_dia'
    return interval

  #Function to create 0_90 credito_vencer interval
  def credito_vencer_interval_0_90(self, column_start, column_middle, column_end, prd):
    column_start = prd + '_' + column_start
    column_middle = prd + '_' + column_middle
    column_end = prd + '_' + column_end
    interval = 'SUM(' + column_start + ' + ' + column_middle + ' + ' + column_end + ') as ' + prd + '_credito_vencer_0_90_dia'
    return interval

  #Function to create 15_60 credito_vencido interval
  def credito_vencido_interval_0_60(self, column_start, column_end, prd):
    column_start = prd + '_' + column_start
    column_end = prd + '_' + column_end
    interval = 'SUM(' + column_start + ' + ' + column_end + ') as ' + prd + '_credito_vencido_15_60_dia'
    return interval

  #Function to create 15_90 credito_vencer interval
  def credito_vencido_interval_0_90(self, column_start, column_middle, column_end, prd):
    column_start = prd + '_' + column_start
    column_middle = prd + '_' + column_middle
    column_end = prd + '_' + column_end
    interval = 'SUM(' + column_start + ' + ' + column_middle + ' + ' + column_end + ') as ' + prd + '_credito_vencido_15_90_dia'
    return interval

  #Update outliers
  def update_outliers(self, min, max, table_columns, table_name):
    min_outlier = str(min)
    max_outlier = str(max)

    for column in table_columns:
      select_median = self.select_median(column, table_name)
      median = str(self.query(self.conn, select_median).values[0]).replace("'", "").replace("[", "").replace("]", "")
      update_query = 'UPDATE ' + table_name + ' ' + \
              'SET ' + column + ' = ' + median + ' ' + \
              'WHERE CAST(' + column + ' AS INTEGER) < ' + min_outlier + ' ' + \
              'OR CAST(' + column + ' AS INTEGER) > ' + max_outlier
      self.query(self.conn, update_query)

  #Update Null and Nan on the table
  def update_null(self, table_columns, table_name):
    for column in table_columns:
      select_median_query = self.select_median(column, table_name)
      median = str(self.query(self.conn, select_median_query).values[0]).replace("'", "").replace("[", "").replace("]", "")
      update_query = 'UPDATE ' + table_name + ' ' + \
              'SET ' + column + ' = ' + median + ' ' + \
              'WHERE ' + column + ' = "nan"' + \
              'OR ' + column + ' = NULL'
      self.query(conn, update_query)

  #Variable to connect
  conn = connect()

# Dictionaries, Lists of producs and List of Columns

In this sction, we'll declare two classes:
1. Products: contains all products infos
2. Table: contains table infos

In [10]:
class Products:
  #Dictionary of products and its respective codes
  dictionary_products = {
    "consignado": 202,
    "cheque_especial": 101,
    "cheque_especial_1": 213,
    "cartao_revolver": 204,
    "cartao_revolver_1": 210,
    "cartao_revolver_2": 218,
    "cartao_revolver_3": 406,
    "cartao_transactor": 1304,
    "outros": 499,
    "credito_pessoal" : 203
    }

  #List of Products
  products = {
    "consignado": 1,
    "cheque_especial": 2,
    "cartao_revolver": 4,
    "cartao_transactor": 1,
    "outros": 1,
    "credito_pessoal": 1
  }

  def get_products_names(self):
    return list(self.products)

  def get_products_qt(self):
    return list(self.products.values())


class Table:
  #Table Name
  table_name = 'scr'

  #List of table columns
  table_columns = [
    "valor_credito_vencer_ate_30_dia",
    "valor_credito_vencer_31_60_dia" ,
    "valor_credito_vencer_61_90_dia" ,
    "valor_credito_vencer_acima_90_dia" ,
    "valor_credito_vencido_15_30_dia" ,
    "valor_credito_vencido_31_60_dia" ,
    "valor_credito_vencido_61_90_dia" ,
    "valor_credito_vencido_acima_90_dia"
  ]

# Declaring Classes and Main Vals

In [11]:
prd = Products()
table = Table()
sql_queries = Query_Builder()

#Product vals
prd_names = prd.get_products_names()
prd_qt = prd.get_products_qt()
prd_dictionary = prd.dictionary_products

#Table vals
table_columns = table.table_columns
table_name = table.table_name
conn = sql_queries.conn

#Application vals
case_query = ''
credito_vencer_0_60_query = ''
credito_vencer_0_90_query = ''
credito_vencido_0_60_query = ''
credito_vencido_0_90_query = ''

# Treating the null and nan

In [12]:
sql_queries.update_null(table_columns, table_name)

# Treating outliers

In [13]:
sql_queries.update_outliers(0, 1000000, table_columns, table_name)

# Final query

In [14]:
#Creaiting Case query
for i in range(0,len(prd_names)):
  case_query += sql_queries.case_query(prd_names[i], prd_dictionary, table_columns, prd_qt[i])  + " , "

case_query = case_query[:-2]

In [15]:
for i in range(0,len(prd_names)):
  credito_vencer_0_60_query += sql_queries.credito_vencer_interval_0_60(table_columns[0], table_columns[1], prd_names[i]) + " , "
  credito_vencer_0_90_query += sql_queries.credito_vencer_interval_0_90(table_columns[0], table_columns[1], table_columns[2], prd_names[i]) + " , "
  credito_vencido_0_60_query += sql_queries.credito_vencido_interval_0_60(table_columns[4], table_columns[5], prd_names[i]) + " , "
  credito_vencido_0_90_query += sql_queries.credito_vencido_interval_0_90(table_columns[4], table_columns[5], table_columns[6], prd_names[i]) + " , "

#Creating the interval credito_vencer 0_60
credito_vencer_0_60_query = credito_vencer_0_60_query[:-2]

#Creating the interval credito_vencer 0_90
credito_vencer_0_90_query = credito_vencer_0_90_query[:-2]

#Creating the interval credito_vencido 0_60
credito_vencido_0_60_query = credito_vencido_0_60_query[:-2]

#Creating the interval credito_vencido 0_90
credito_vencido_0_90_query = credito_vencido_0_90_query[:-2]

In [16]:
select_credit_modeling = """
    SELECT chave_cpf, data_consulta_dado_bacen,	""" + \
      sql_queries.create_SUM(prd_names, table_columns) + \
      ' , ' + \
      credito_vencer_0_60_query + \
      ' , ' + \
      credito_vencer_0_90_query + \
      ' , ' + \
      credito_vencido_0_60_query + \
      ' , ' + \
      credito_vencido_0_90_query + \
    """
    FROM
      (
      SELECT *, 
      """ + \
        case_query + \
      'FROM ' + table_name + ' ' + \
      """
      )
    GROUP BY chave_cpf, data_consulta_dado_bacen
"""

df_credit_modeling = sql_queries.query(conn, select_credit_modeling)

In [18]:
df_credit_modeling.describe()

,consignado_valor_credito_vencer_ate_30_dia,consignado_valor_credito_vencer_31_60_dia,consignado_valor_credito_vencer_61_90_dia,consignado_valor_credito_vencer_acima_90_dia,consignado_valor_credito_vencido_15_30_dia,consignado_valor_credito_vencido_31_60_dia,consignado_valor_credito_vencido_61_90_dia,consignado_valor_credito_vencido_acima_90_dia,cheque_especial_valor_credito_vencer_ate_30_dia,cheque_especial_valor_credito_vencer_31_60_dia,...,cartao_revolver_credito_vencido_15_60_dia,cartao_transactor_credito_vencido_15_60_dia,outros_credito_vencido_15_60_dia,credito_pessoal_credito_vencido_15_60_dia,consignado_credito_vencido_15_90_dia,cheque_especial_credito_vencido_15_90_dia,cartao_revolver_credito_vencido_15_90_dia,cartao_transactor_credito_vencido_15_90_dia,outros_credito_vencido_15_90_dia,credito_pessoal_credito_vencido_15_90_dia
count,87522.000000,87522.000000,87522.000000,87522.000000,87522.000000,87522.000000,87522.000000,87522.000000,87522.000000,87522.00000,...,87522.000000,87522.000000,87522.000000,87522.000000,87522.000000,87522.000000,87522.000000,87522.000000,87522.000000,87522.000000
mean,224.497693,211.606947,209.821344,9248.503742,7.260535,5.393253,6.920519,25.161242,163.052181,28.25506,...,226.026952,0.207481,0.292105,52.025690,19.574308,28.345351,379.827637,0.231726,0.389989,79.214890
std,689.688385,629.200504,604.575361,31734.538976,93.379068,97.328743,322.508798,587.157244,1563.179026,629.10157,...,2043.756688,23.323723,12.870844,546.484426,376.408092,755.414740,3244.970905,24.324476,17.832775,840.215608
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,111.030000,111.010000,115.457500,3130.685000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,23673.480000,20700.260000,20212.320000,964910.210000,5648.680000,18744.560000,66820.680000,63433.650000,142316.330000,76177.25000,...,283044.610000,6180.980000,1750.990000,56511.910000,66820.680000,104783.460000,367975.200000,6180.980000,2488.550000,98346.140000


Saving the DF in csv

In [19]:
df_credit_modeling.to_csv('book_scr.csv', index=False)